In [2]:
import os
import random
import time
from distutils.util import strtobool
# Load the TensorBoard notebook extension
%load_ext tensorboard


import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

In [3]:

# Clear any logs from previous runs
import tensorflow as tf
import datetime


mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])
%tensorboard --logdir logs/fit



Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2207 - accuracy: 0.9347 - val_loss: 0.1030 - val_accuracy: 0.9690
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0973 - accuracy: 0.9703 - val_loss: 0.0899 - val_accuracy: 0.9705
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0687 - accuracy: 0.9790 - val_loss: 0.0723 - val_accuracy: 0.9784
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0527 - accuracy: 0.9829 - val_loss: 0.0697 - val_accuracy: 0.9782
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0434 - accuracy: 0.9857 - val_loss: 0.0717 - val_accuracy: 0.9773


In [3]:
def make_env(gym_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(gym_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ClipRewardEnv(env)
        env = gym.wrappers.ResizeObservation(env, (84, 84))
        env = gym.wrappers.GrayScaleObservation(env)
        env = gym.wrappers.FrameStack(env, 4)
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk


def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer